## Pre-Processing Data

### Importing all required libraries

In [ ]:
pip install scipy

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime as dt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score,r2_score,mean_absolute_error,mean_squared_error,accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict
from sklearn import svm,metrics,tree,preprocessing,linear_model
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score,mean_squared_error,recall_score,confusion_matrix,f1_score,roc_curve, auc
from plotly.offline import iplot, init_notebook_mode
import pickle
import warnings
warnings.filterwarnings("ignore") 
import datetime as dt
from datetime import datetime
import plotly.express as px
import ortools.constraint_solver
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from scipy.optimize import linprog
from pylab import rcParams
import scipy
from scipy.stats.stats import pearsonr
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import utils
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import classification_report

In [ ]:
MPSCData=pd.read_csv("/kaggle/input/supply-chain-analysis/supply_chain_data.csv")

## Descriptive Part

In [ ]:
MPSCData.shape  #Checking the lengths of the array dimensions

In [ ]:
MPSCData.dtypes  #Checking the number of variables and their type

***

In [ ]:
MPSCData.head(10)  #Checking the top 10 rows in the dataset

***

In [ ]:
MPSCData.info()  #Checking the type of column structure

***

In [ ]:
MPSCData.describe()  # Returning statistical description of the data in the Dataset

A brief explanation of the values in the table above:

Since the target variables are "Stock levels" and "Route Costs", their descriptive statistics indicators are significant.
1)The average of the "Stock level" is 47.77, with a minimum of 0, and a maximum of 100.0.  This variable has a very wide Range, so its values are more dispersed.
As the Second Quantile (Q2) is the Median, the median value is 47.50, which is almost the same as the mean, so the data are normally distributed (symmetric).

Also, the minimum stock level is an alarm for the company that is facing shortage of inventory for that SKU.

2)The average of the "Route Costs" is 529.25, with a minimum of 103.92, and a maximum of 997.41. This variable has a very wide Range.
As the Second Quantile (Q2) is the Median, the median value is 520.43, almost equal to the mean, so the data are normally distributed (symmetric).

## Data Cleaning

### Missing Values

In [ ]:
np.sum(MPSCData.isna()) #Checking the number of missing values for each variable

As can be seen, there are no missing values in this dataset.

### Duplicate Values

In [ ]:
MPSCData.duplicated().sum()

This dataset also has no duplicate values.

### Cleaning

In [ ]:
MPSCData.columns = [col.lower().replace(' ', '_') for col in MPSCData.columns]
MPSCData.rename(columns=lambda x: x.replace("(", "").replace(")", ""), inplace=True)

In [ ]:
MPSCData.columns

***********

## Correlation Matrix

 In the business world, knowing the relationship between two variables is very valuable for data-driven decision-making.  In this case, the "correlation coefficient" can be used to calculate the relationship between two quantitative variables. 

To examine the relationship between the product type, customer demographics, shipping carriers, location, transportation modes, inspection results, routes, and the two target variables of inventory level and transport cost, we must first convert these object characteristics into the "int" type using the following library and code:

In [ ]:
MP_SC = MPSCData.copy()

In [ ]:
le = preprocessing.LabelEncoder()# create the Labelencoder object
MP_SC['product_type']= le.fit_transform(MP_SC['product_type'])#convert the categorical columns into numeric
MP_SC['customer_demographics']= le.fit_transform(MP_SC['customer_demographics'])
MP_SC['shipping_carriers']= le.fit_transform(MP_SC['shipping_carriers'])
MP_SC['location']= le.fit_transform(MP_SC['location'])
MP_SC['sku']= le.fit_transform(MP_SC['sku'])
MP_SC['inspection_results']= le.fit_transform(MP_SC['inspection_results'])
MP_SC['transportation_modes']= le.fit_transform(MP_SC['transportation_modes'])
MP_SC['routes']= le.fit_transform(MP_SC['routes'])
MP_SC['supplier_name']= le.fit_transform(MP_SC['supplier_name'])

In [ ]:
SC_features=MP_SC[['product_type','price',
       'number_of_products_sold', 'revenue_generated', 'customer_demographics', 'lead_times', 'order_quantities', 'shipping_times',
        'shipping_costs', 'location', 'stock_levels',
       'lead_time', 'production_volumes', 'manufacturing_lead_time',
       'manufacturing_costs', 'inspection_results', 'defect_rates',
       'transportation_modes', 'routes', 'costs']]
fig = plt.figure(figsize=(20,10))
sns.heatmap(SC_features.corr(), annot = True, fmt = '.2f', cmap = "RdYlGn")

In [ ]:
%matplotlib inline
rcParams['figure.figsize']=5,4
sb.set_style('whitegrid')

In [ ]:
MP=MP_SC[['product_type','price',
       'number_of_products_sold', 'revenue_generated', 'customer_demographics', 'lead_times', 'order_quantities', 'shipping_times',
        'shipping_costs', 'location', 'stock_levels',
       'lead_time', 'production_volumes', 'manufacturing_lead_time',
       'manufacturing_costs', 'inspection_results', 'defect_rates',
       'transportation_modes', 'routes', 'costs']]

In [ ]:
sb.pairplot(MP)

In [ ]:
STOCK=MP_SC[['price',
       'number_of_products_sold', 'revenue_generated', 'order_quantities',
        'shipping_costs', 'stock_levels',
       'lead_time', 'production_volumes', 'manufacturing_lead_time',
       'manufacturing_costs', 'defect_rates']]
sb.pairplot(STOCK)

In [ ]:
STOCK=MP_SC[['product_type','price',
       'number_of_products_sold', 'revenue_generated', 'customer_demographics', 'lead_times', 'order_quantities', 'shipping_times',
        'shipping_costs', 'location', 'stock_levels',
       'lead_time', 'production_volumes', 'manufacturing_lead_time',
       'manufacturing_costs', 'inspection_results', 'defect_rates',
       'transportation_modes', 'routes', 'costs']]

In [ ]:
corr=STOCK.corr()
corr

In [ ]:
COSTS=MP_SC[['price',
       'number_of_products_sold', 'revenue_generated', 'lead_times', 'order_quantities',
        'shipping_costs', 'stock_levels',
       'lead_time', 'production_volumes', 'manufacturing_lead_time',
       'manufacturing_costs', 'defect_rates', 'costs']]
sb.pairplot(COSTS)

In [ ]:
COSTS=MP_SC[['product_type','price',
       'number_of_products_sold', 'revenue_generated', 'customer_demographics', 'lead_times', 'order_quantities', 'shipping_times',
        'shipping_costs', 'location', 'stock_levels',
       'lead_time', 'production_volumes', 'manufacturing_lead_time',
       'manufacturing_costs', 'inspection_results', 'defect_rates',
       'transportation_modes', 'routes', 'costs']]

In [ ]:
corr=COSTS.corr()
corr

****

## Data Visualisation

***GroupBy:***

Product type, Routes, Customer demographics, Shipping carrier, Location, and Transportation modes are divided into distinct groups using the "group by" method, and in each category, the key elements like Revenue, Costs, Stock levels, and Order quantities are examined.

In [ ]:
Product = MPSCData.groupby('product_type') 
Route = MPSCData.groupby('routes')
Customer=MPSCData.groupby('customer_demographics')
Shipping=MPSCData.groupby('shipping_carriers')
Location=MPSCData.groupby('location')
Transportation=MPSCData.groupby('transportation_modes')

****

### Products Analysis

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(4, 2, 1)
Product['stock_levels'].sum().sort_values(ascending=False).plot.bar(figsize=(12,14), title="Total Stock")

plt.subplot(4, 2, 2)
Product['order_quantities'].sum().sort_values(ascending=False).plot.bar(figsize=(12,14), title="Total Order")

plt.subplot(4, 2, 3)
Product['manufacturing_costs'].sum().sort_values(ascending=False).plot.bar(figsize=(12,14), title="Manufacturing Costs")

plt.subplot(4, 2, 4)
Product['revenue_generated'].sum().sort_values(ascending=False).plot.bar(figsize=(12,14), title="Revenue")

plt.tight_layout()
plt.show()

data_Products=MPSCData.groupby(['product_type'])['sku'].count().reset_index(name='number_of_products_sold').sort_values(by= 'number_of_products_sold', ascending= False)
px.pie(data_Products, values='number_of_products_sold', names= 'product_type' , title= 'Total Number of Products Sold', 
       color='product_type',
             color_discrete_map={'cosmetics':'skyblue',
                                 'haircare':'navajowhite',
                              'skincare':'lawngreen'})



***

### Customers Analysis

In [ ]:
data_Customers=MPSCData.groupby(['customer_demographics'])['sku'].count().reset_index(name='number_of_products_sold').sort_values(by= 'number_of_products_sold', ascending= False)
px.pie(data_Customers, values='number_of_products_sold', names= 'customer_demographics' , title= 'Customer Segment', 
       color='customer_demographics',
             color_discrete_map={'Female':'lightgreen',
                                 'Male':'blue',
                                 'Non-binary':'orange',
                              'Unknown':'crimson'})


### Customer Segment by Products

In [ ]:
Customer_Segment_by_Products= MPSCData.groupby(["customer_demographics","product_type"])["sku"].count().reset_index()
Customer_Segment_by_Products

In [ ]:
bar_Customer_Segment_by_Products = px.bar(Customer_Segment_by_Products, x='customer_demographics', y='sku', \
    title='Customer Segment by Products',color='product_type')
bar_Customer_Segment_by_Products.show()

### Routes Analysis

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(4, 2, 1)
Route['revenue_generated'].sum().sort_values(ascending=False).plot.bar(figsize=(12,14), title="Revenue")

plt.subplot(4, 2, 2)
Route['order_quantities'].sum().sort_values(ascending=False).plot.bar(figsize=(12,14), title="Total Order")

plt.subplot(4, 2, 3)
Route['costs'].sum().sort_values(ascending=False).plot.bar(figsize=(12,14), title="Tranportation Costs")

plt.subplot(4, 2, 4)

Route['shipping_times'].mean().sort_values(ascending=False).plot.bar(figsize=(12,14), title="Avg.Shipping Time")

plt.tight_layout()
plt.show()


data_Routes=MPSCData.groupby(['routes'])['sku'].count().reset_index(name='number_of_products_sold').sort_values(by= 'number_of_products_sold', ascending= False)
px.pie(data_Routes, values='number_of_products_sold', names= 'routes' , title= 'Routes', 
       color='routes',
             color_discrete_map={'Route A':'purple',
                                 'Route B':'lime',
                                 'Route C':'bisque'})

### Transportation Modes

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(4, 2, 1)
Transportation['revenue_generated'].sum().sort_values(ascending=False).plot.bar(figsize=(12,14), title="Revenue")

plt.subplot(4, 2, 2)
Transportation['order_quantities'].sum().sort_values(ascending=False).plot.bar(figsize=(12,14), title="Total Order")

plt.subplot(4, 2, 3)
Transportation['costs'].sum().sort_values(ascending=False).plot.bar(figsize=(12,14), title="Tranportation Costs")

plt.subplot(4, 2, 4)

Transportation['shipping_times'].mean().sort_values(ascending=False).plot.bar(figsize=(12,14), title="Avg. Shipping Time")

plt.tight_layout()
plt.show()



data_Transportation=MPSCData.groupby(['transportation_modes'])['sku'].count().reset_index(name='number_of_products_sold').sort_values(by= 'number_of_products_sold', ascending= False)
px.pie(data_Transportation, values='number_of_products_sold', names= 'transportation_modes' , title= 'Transportation Modes', 
       color='transportation_modes',
             color_discrete_map={'Air':'deepskyblue',
                                 'Rail':'burlywood',
                                 'Road':'yellowgreen',
                              'Sea':'aquamarine'})

### Routes and Transportation modes

In [ ]:
Routes_by_Transportation= MPSCData.groupby(["routes","transportation_modes"])["sku"].count().reset_index()
Routes_by_Transportation

In [ ]:
bar_Routes_by_Transportation = px.bar(Routes_by_Transportation, x='routes', y='sku', \
    title='Routes_by_Transportation Modes',color='transportation_modes')
bar_Routes_by_Transportation.show()

******

## Data Modelling

Data modelling in supply chain management allows a company to respond quickly to market trends, adapt production, and optimise logistics to ensure optimal supply chain performance. Consequently, a company will be able to maximise production, optimise transportation, and save money. Furthermore, the company can make data-driven decisions and take proactive measures to meet customer expectations, streamline processes, and optimise resources (Siththardhan, 2023).

### Regression Models

Regression models can identify patterns, recognize demand indications, and identify correlations between variables in large datasets. According to [McKinsey](#4) ML-based supply chain solutions can cut prediction mistakes by up to 50%.

In [ ]:
train_MPSC = MP_SC.copy()

#### Linear Regression for *Stock Level*

In [ ]:
train_MPSC['lead_times']=train_MPSC['lead_times'].astype(int)
train_MPSC['shipping_times']= train_MPSC['shipping_times'].astype(int)
train_MPSC['lead_time']= train_MPSC['lead_time'].astype(int)
train_MPSC['manufacturing_lead_time']= train_MPSC['manufacturing_lead_time'].astype(int)

In [ ]:
X=train_MPSC[[ 'sku', 'price',
        'revenue_generated',
        'lead_times', 'shipping_times', 'shipping_costs',
       'lead_time', 'production_volumes', 'manufacturing_lead_time',
       'manufacturing_costs', 'inspection_results', 'defect_rates', 'routes', 'costs']]

Y=train_MPSC[['stock_levels']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()# Scale the data 
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train a linear regression model
model = LinearRegression()
model.fit(X_train_scaled, Y_train)

In [ ]:
# Evaluate the model on the testing set
Y_pred = model.predict(X_test_scaled)
r2 = r2_score(Y_test, Y_pred)
print(f"R-squared: {r2}")


Cohen (1988) suggested R2 values for endogenous latent variables are assessed as follows: 0.26 (substantial), 0.13 (moderate), 0.02 (weak).

In [ ]:
# Create a scatter plot with the actual Stocks Levels on the x-axis and the predicted Stocks Levels values on the y-axis
plt.scatter(Y_test, Y_pred)

# Plot a regression line to see how well the model has fit the data
sns.regplot(x=Y_test, y=Y_pred)

plt.xlabel('Actual Stocks Levels')
plt.ylabel('Predicted Stocks Levels')
plt.title('Predicted vs Actual Stocks Levels')
plt.show()

#### Linear Regression for *Transportation Costs*

In [ ]:
train_MPSC['Intercept'] = 1
X=train_MPSC[['Intercept', 'sku', 'lead_times', 'order_quantities', 'location']]

Y=train_MPSC[['costs']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()# Scale the data 
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train a linear regression model
model = LinearRegression()
model.fit(X_train_scaled, Y_train)

In [ ]:
# Evaluate the model on the testing set
Y_pred = model.predict(X_test_scaled)
r2 = r2_score(Y_test, Y_pred)
ols_model = sm.OLS(MP_SC['costs'], X)
results = ols_model.fit()
results.summary()

1-I examined the p-value for predictor variables. To decide on the retention of a predictor variable in the model, its p-value must be less than 0.05 which means that the test hypothesis is false or should be rejected.

2-If the p-value is less than 0.05, it can be said that there is a statistically significant relationship between these predictor variables and the response variable.  

3-If the p-value is greater than 0.05 for the predictor variables, it means that no effect was observed. So this variable does not affect the response variable. 

4-I performed the OLS regression model again by removing the predictor variables that had a p-value above 0.05.

In [ ]:
# Create a scatter plot with the actual Transportation Costs on the x-axis and the predicted Transportation Costs values on the y-axis
plt.scatter(Y_test, Y_pred)

# Plot a regression line to see how well the model has fit the data
sns.regplot(x=Y_test, y=Y_pred)

plt.xlabel('Actual Transportation Costs')
plt.ylabel('Predicted Transportation Costs')
plt.title('Predicted vs Actual Transportation Costs')
plt.show()

******

### Classification Models

In machine learning, classification is the process of predicting a categorical label for a given input point. A trained algorithm uses input features to map inputs to class labels (datacamp, 2023).

#### KNN Classifier for Routes

KNN is the most basic classification algorithm in Machine Learning. It belongs to the supervised learning domain and is used extensively for pattern recognition, data mining, and intrusion detection." The KNN algorithm is non-parametric, meaning it does not assume anything about the distribution of the data and the model structure is determined entirely by the data. To predict the classification of a new sample point based on the K (integer number) closest points in a database, it uses a database in which the data points are separated into several classes. We use the k-nearest neighbour algorithm to fit historical data and predict the future.

In [ ]:
train_MP=MP_SC.copy()

In [ ]:
sns.scatterplot(x=train_MP['sku'],y=train_MP['costs'], hue=train_MP['routes'])

I use cross-validation to find the accuracy scores, which means the data needs to scale, then loop over the values and add the scores to the list.
I pass an instance of the kNN model as well as the data and several splits to be performed. I use five splits, which means that the model divides the data into five equal-sized groups, four of which are used to train and one to test the outcome. It will loop through each group and give an accuracy score, which it will average to find the best model.

In [ ]:
# Create feature and target arrays
X = train_MP[['product_type', 'sku', 'price', 'availability', 'number_of_products_sold', 'revenue_generated', 'customer_demographics', 'stock_levels', 'lead_times', 'order_quantities', 'shipping_times', 'shipping_carriers', 'shipping_costs', 'supplier_name', 'location', 'lead_time', 'production_volumes', 'manufacturing_lead_time', 'manufacturing_costs', 'inspection_results', 'defect_rates', 'transportation_modes', 'costs']]
y = train_MP[['routes']]
 
    
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size = 0.2, random_state=42)


neighbors = np.arange(1, 31)
train_accuracy = np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))
  
# Loop over K values
for i, k in enumerate(neighbors):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
      
    # Compute training and test data accuracy
    train_accuracy[i] = knn.score(X_train, y_train)
    test_accuracy[i] = knn.score(X_test, y_test)
  
# Generate plot
plt.plot(neighbors, test_accuracy, label = 'Testing dataset Accuracy', color="purple")
plt.plot(neighbors, train_accuracy, label = 'Training dataset Accuracy', color="green")
  
plt.legend()
plt.xlabel('n_neighbors')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
#Setup a knn classifier with k neighbors
knn = KNeighborsClassifier(n_neighbors=30)
 
#Fit the model
knn.fit(X_train,y_train) 
#Get accuracy. 
#Note: In case of classification algorithms score method 
#represents accuracy. 
knn.score(X_test,y_test)


In [ ]:
X = train_MP[['product_type', 'sku', 'price', 'availability', 'number_of_products_sold', 'revenue_generated', 'customer_demographics', 'stock_levels', 'lead_times', 'order_quantities', 'shipping_times', 'shipping_carriers', 'shipping_costs', 'supplier_name', 'location', 'lead_time', 'production_volumes', 'manufacturing_lead_time', 'manufacturing_costs', 'inspection_results', 'defect_rates', 'transportation_modes', 'costs']]
y = train_MP[['routes']]

  
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size = 0.2, random_state=42)
  
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
  
  # Calculate the accuracy of the model
y_pred = knn.predict(X_test)
print(confusion_matrix(y_test, y_pred))

****

In [ ]:
k_values = [i for i in range (1,31)]
scores = []

scaler = StandardScaler()
X = scaler.fit_transform(X)

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    score = cross_val_score(knn, X, y, cv=5)
    scores.append(np.mean(score))


In [ ]:
sns.lineplot(x = k_values, y = scores, marker = 'o')
plt.xlabel("K Values")
plt.ylabel("Accuracy Score")